# Teacher Annotation Analysis

Qwen2.5-72B-Instruct reasoning traces on COVID-19 tweets.

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
from pathlib import Path
import glob

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_colwidth', 150)

## Load Annotated Data

In [ ]:
annotation_files = glob.glob("data/COVIDSenti/COVIDSenti_conversational_annotated_*.csv")

if not annotation_files:
    print("No annotation files found")
else:
    annotation_files.sort(key=lambda x: Path(x).stat().st_size, reverse=True)
    file_path = annotation_files[0]
    
    df = pd.read_csv(file_path)
    print(f"Loaded: {file_path}")
    print(f"Samples: {len(df):,}")

## Dataset Overview

In [ ]:
print(f"Total samples: {len(df):,}")
print(f"\nSentiment distribution:")
print(df['label'].value_counts())
print(f"\nSample tweet:")
print(f"  {df.iloc[0]['tweet'][:100]}...")

## Sentiment Distribution

In [ ]:
label_counts = df['label'].value_counts()
label_names = {'neu': 'Neutral', 'neg': 'Negative', 'pos': 'Positive'}
label_colors = {'neu': '#95a5a6', 'neg': '#e74c3c', 'pos': '#2ecc71'}

display_labels = [label_names.get(lbl, lbl) for lbl in label_counts.index]
colors = [label_colors.get(lbl, '#3498db') for lbl in label_counts.index]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

label_counts.plot(kind='bar', ax=ax1, color=colors)
ax1.set_title('Sentiment Distribution')
ax1.set_xlabel('Sentiment')
ax1.set_ylabel('Count')
ax1.set_xticklabels(display_labels, rotation=0)

label_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=colors, 
                   labels=display_labels, startangle=90)
ax2.set_ylabel('')
ax2.set_title('Proportion')

plt.tight_layout()
plt.show()

## Browse Annotations

In [ ]:
def display_annotation(idx):
    if idx >= len(df):
        print(f"Index {idx} out of range")
        return
    
    row = df.iloc[idx]
    
    output = f"""
# Example {idx + 1} / {len(df)}

## Tweet
**"{row['tweet']}"**

**Label:** `{row['label'].upper()}`

---

## R1: Syntactic Parsing
{row['r1_syntactic']}

---

## R2: Aspect Extraction
{row['r2_aspects']}

---

## R3: Opinion Extraction
{row['r3_opinion']}

---

## R4: Sentiment Classification
{row['r4_sentiment']}

---

## R5: Emotion Classification
{row['r5_emotion']}
"""
    display(Markdown(output))

display_annotation(0)

## Random Example

In [ ]:
import random
random_idx = random.randint(0, len(df)-1)
display_annotation(random_idx)

## Filter by Sentiment

In [ ]:
sentiment = 'neg' #neg pos or neu

sentiment_df = df[df['label'] == sentiment]
print(f"Found {len(sentiment_df):,} tweets with '{sentiment.upper()}' sentiment")

if len(sentiment_df) > 0:
    original_idx = sentiment_df.index[0]
    display_annotation(original_idx)

## Reasoning Trace Statistics

In [ ]:
reasoning_cols = ['r1_syntactic', 'r2_aspects', 'r3_opinion', 'r4_sentiment', 'r5_emotion']
task_names = ['R1: Syntactic', 'R2: Aspects', 'R3: Opinion', 'R4: Sentiment', 'R5: Emotion']

lengths = []
for col in reasoning_cols:
    df[f'{col}_len'] = df[col].str.len()
    lengths.append(df[f'{col}_len'].mean())

print("Average reasoning trace lengths:")
for name, length in zip(task_names, lengths):
    print(f"  {name:20s}: {length:6.0f} characters")

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(task_names, lengths, color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'])
ax.set_title('Average Reasoning Trace Length by Task')
ax.set_xlabel('Task')
ax.set_ylabel('Characters')
plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

## Export Samples

In [ ]:
sample_df = df.sample(n=min(10, len(df)), random_state=42)
output_path = "data/COVIDSenti/sample_annotations_for_review.csv"
sample_df.to_csv(output_path, index=False)
print(f"Exported {len(sample_df)} samples to {output_path}")